In [1]:
import numpy as np

In [2]:
identifier = 'LSTMGAN-BS56LR0.01GR10SeqLen30Sepsis'

In [3]:
real_data_Sepsis = np.load('./data/'+identifier+'real_data.npy')

In [4]:
fake_data_Sepsis = np.load('./data/'+identifier+'fake_data.npy')

In [5]:
real_data_Sepsis.shape

(2627, 30, 5)

In [6]:
fake_data_Sepsis.shape

(2184, 30, 5)

In [7]:
identifier = 'LSTMGAN-BS56LR0.01GR10SeqLen30-Non-Sepsis'

In [8]:
real_data_NonSepsis = np.load('./data/'+identifier+'real_data.npy')

In [9]:
fake_data_NonSepsis = np.load('./data/'+identifier+'fake_data.npy')

In [10]:
real_data_NonSepsis.shape

(1314, 30, 5)

In [11]:
fake_data_NonSepsis.shape

(1064, 30, 5)

## Train on Synthetic

In [12]:
## 0 - Non-Sepsis
## 1 - Sepsis

In [1]:
#fake_data_NonSepsis[0]

In [2]:
#fake_data_Sepsis[0]

In [15]:
fake_data_X = np.vstack((fake_data_NonSepsis,fake_data_Sepsis))
fake_data_X.shape

(3248, 30, 5)

In [16]:
fake_data_Y = np.concatenate([np.zeros(fake_data_NonSepsis.shape[0]),np.ones(fake_data_Sepsis.shape[0])])
fake_data_Y.shape

(3248,)

In [17]:
seq_len = real_data_Sepsis.shape[1]
feature_len = real_data_Sepsis.shape[2]

In [18]:
print(seq_len)
print(feature_len)

30
5


## Model

In [19]:
import tensorflow as tf

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [21]:
model = Sequential()
model.add(LSTM(100, input_shape=(seq_len, feature_len)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 42,501
Trainable params: 42,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_val, y_train, y_val = train_test_split(fake_data_X, fake_data_Y, test_size=0.2, random_state=42)

In [25]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(2598, 30, 5)
(650, 30, 5)
(2598,)
(650,)


In [26]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

Train on 2598 samples, validate on 650 samples
Epoch 1/5
2598/2598 [==============================] - 31s 12ms/step - loss: 0.1619 - acc: 0.9226 - val_loss: 0.0340 - val_acc: 0.9938
Epoch 2/5
2598/2598 [==============================] - 20s 8ms/step - loss: 0.0086 - acc: 0.9977 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 3/5
2598/2598 [==============================] - 20s 8ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 4.8088e-04 - val_acc: 1.0000
Epoch 4/5
2598/2598 [==============================] - 21s 8ms/step - loss: 0.0034 - acc: 0.9996 - val_loss: 5.8368e-04 - val_acc: 1.0000
Epoch 5/5
2598/2598 [==============================] - 21s 8ms/step - loss: 2.4448e-04 - acc: 1.0000 - val_loss: 1.8616e-04 - val_acc: 1.0000


## Test on Real

In [27]:
real_data_X = np.vstack((real_data_NonSepsis,real_data_Sepsis))
real_data_X.shape

(3941, 30, 5)

In [28]:
real_data_Y = np.concatenate([np.zeros(real_data_NonSepsis.shape[0]),np.ones(real_data_Sepsis.shape[0])])
real_data_Y.shape

(3941,)

In [29]:
scores = model.evaluate(real_data_X, real_data_Y, verbose=1)

3941/3941 [==============================] - 8s 2ms/step


In [32]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 57.60%
